my code to do multitaper spec for linear spectrogram scaling, above code is aaron's. Delete his later. 4/5.

In [ ]:
import os
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne.time_frequency

from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, crop_empty_data, outliers_to_nan, channel_outlier_marker
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import crop_pad
import numpy as np

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

subjects = layout.get(return_type="id", target="subject")


print(subjects)

subjects = ['D0100', 'D0102', 'D0103'] # JUST DO ONE FOR THE POSTER CUZ THIS USES SO MUCH DAMN MEMORY

for sub in subjects:
        # Load the data
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                                extension='.edf', desc='clean', preload=False) #get line-noise filtered data
        print(filt)

        ## Crop raw data to minimize processing time
        good = crop_empty_data(filt)

        # good.drop_channels(good.info['bads'])
        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        good.drop_channels(good.info['bads'])
        # good.info['bads'] += channel_outlier_marker(good, 4, 2)
        # good.drop_channels(good.info['bads'])
        good.load_data()

        ch_type = filt.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Remove intermediates from mem
        good.plot()

        ## epoching and trial outlier removal

        save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'multitaper_alt', sub)
        if not os.path.exists(save_dir):
                os.makedirs(save_dir)

        # make stimulus baseline EpochsTFR
        times = [-0.7, 0.5] #padd 0.5 secs each side, want 0.2 seconds before stim onset
        print("times: " + str(times))
        trials = trial_ieeg(good, "Stimulus", times, preload=True)

        # print("base trials shape is: " + str(trials._data.shape))
        outliers_to_nan(trials, outliers=10)
        freq = np.arange(10, 200., 2.)
        kwargs = dict(average=False, n_jobs=1, freqs=freq, return_itc=False,
                      n_cycles=freq / 2, time_bandwidth=10, decim=int(good.info['sfreq'] / 100))
        
        # del good
        # del filt

        base = mne.time_frequency.tfr_multitaper(trials, **kwargs)
        crop_pad(base, "0.5s")
        base = base.average(lambda x: np.nanmean(x, axis=0), copy=False)

        #now do rescale with the concatenated baseline epochs
        for event, t in [("Stimulus",(-1, 2))]:
                times = [None, None]
                times[0] = t[0] - 0.5
                times[1] = t[1] + 0.5
                trials = trial_ieeg(good, event, times, preload=True)
                outliers_to_nan(trials, outliers=10)
                spectra = mne.time_frequency.tfr_multitaper(trials,  **kwargs)
                crop_pad(spectra, "0.5s")
                del trials
                spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=False)
                rescale(spectra._data, base._data, mode='ratio', axis=-1)
                fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
                spectra.info['subject_info']['files'] = tuple(fnames)
                spectra.info['bads'] = good.info['bads']
                filename = os.path.join(save_dir, f'{event}_fixationCrossBase_0.2sec-tfr.h5')
                mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
                spectra.save(os.path.join(save_dir, f'{event}_fixationCrossBase_0.2sec-avg.fif'), overwrite=True)


actually plot them

In [ ]:
## Description: Check channels for outliers and remove them
import mne.time_frequency
from ieeg.viz.ensemble import chan_grid
from ieeg.viz.parula import parula_map
from ieeg.io import get_data, update, get_bad_chans
import os

## check if currently running a slurm job
HOME = os.path.expanduser("~")
if 'SLURM_ARRAY_TASK_ID' in os.environ.keys():
    LAB_root = os.path.join(HOME, "workspace", "CoganLab")
    subject = int(os.environ['SLURM_ARRAY_TASK_ID'])
else:  # if not then set box directory
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    subject = 57


# Load the data
TASK = "GlobalLocal"
layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
conds = ["Stimulus_fixationCrossBase_0.2sec"]
subjects = ['D0094']
for subj in subjects:
    for cond in conds:
        spec_type = 'multitaper_alt'
        filename = os.path.join(layout.root, 'derivatives',
                                'spec', spec_type, subj, f'{cond}-tfr.h5')
        spec = mne.time_frequency.read_tfrs(filename)
        info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])
        all_bad = get_bad_chans(info_file)
        spec.info.update(bads=[b for b in all_bad if b in spec.ch_names])

        ## plotting
        import matplotlib as mpl
        figs = chan_grid(spec, size=(20, 10), vmin=0.7, vmax=1.4,
                         cmap=parula_map, show=False)
        fig_path = os.path.join(layout.root, 'derivatives', 'figs', spec_type)
            # Ensure the directory exists
        os.makedirs(fig_path, exist_ok=True)
        for i, f in enumerate(figs):
            f.savefig(os.path.join(fig_path, f'{subj}_{cond}_{i + 1}.jpg'), bbox_inches='tight')
            f.clf()

        ## save bad channels
        # update(spec, layout, "muscle")

aarons code

In [ ]:
## Description: Check channels for outliers and remove them
import mne.time_frequency
from ieeg.viz.ensemble import chan_grid
from ieeg.viz.parula import parula_map
from ieeg.io import get_data, update, get_bad_chans
import os

## check if currently running a slurm job
HOME = os.path.expanduser("~")
if 'SLURM_ARRAY_TASK_ID' in os.environ.keys():
    LAB_root = os.path.join(HOME, "workspace", "CoganLab")
    subject = int(os.environ['SLURM_ARRAY_TASK_ID'])
else:  # if not then set box directory
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    subject = 57


# Load the data
TASK = "SentenceRep"
layout = get_data("SentenceRep", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
conds = ["aud_ls", "aud_lm", "go_ls", "go_lm", "resp"]

for subj in subjects:
    for cond in conds:
        spec_type = 'multitaper'
        filename = os.path.join(layout.root, 'derivatives',
                                'spec', spec_type, subj, f'{cond}-tfr.h5')
        spec = mne.time_frequency.read_tfrs(filename)[0]
        info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])
        all_bad = get_bad_chans(info_file)
        spec.info.update(bads=[b for b in all_bad if b in spec.ch_names])

        ## plotting
        import matplotlib as mpl
        figs = chan_grid(spec, size=(20, 10), vmin=-2, vmax=2,
                         cmap=parula_map, show=False)
        fig_path = os.path.join(layout.root, 'derivatives', 'figs', spec_type)
        for i, f in enumerate(figs):
            f.savefig(os.path.join(fig_path, f'{subj}_{cond}_{i + 1}.jpg'), bbox_inches='tight')
            f.clf()

        ## save bad channels
        # update(spec, layout, "muscle")

just plot one channel for the poster with a colorbar 4/8

In [ ]:
## Description: Check channels for outliers and remove them
import mne.time_frequency
from ieeg.viz.utils import chan_grid
from ieeg.viz.parula import parula_map
from ieeg.io import get_data, update, get_bad_chans
import os
import matplotlib.pyplot as plt

# Set the default font size to 24 for all text in all plots
plt.rcParams.update({'font.size': 24})

## check if currently running a slurm job
HOME = os.path.expanduser("~")
if 'SLURM_ARRAY_TASK_ID' in os.environ.keys():
    LAB_root = os.path.join(HOME, "workspace", "CoganLab")
    subject = int(os.environ['SLURM_ARRAY_TASK_ID'])
else:  # if not then set box directory
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    subject = 57
    pick = 'RMMF14'


# Load the data
TASK = "GlobalLocal"
layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
conds = ["Stimulus_fixationCrossBase_0.2sec"]
subjects = ['D0057']
pick = 'RAI7'


for subj in subjects:
    if int(subj[1:]) != subject:
        continue
    for cond in conds:
        spec_type = 'multitaper'
        filename = os.path.join(layout.root, 'derivatives',
                                'spec', spec_type, subj, f'{cond}-tfr.h5')
        try:
            spec = mne.time_frequency.read_tfrs(filename)[0]
        except OSError:
            continue
        info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])
        all_bad = get_bad_chans(info_file)
        spec.info.update(bads=[b for b in all_bad if b in spec.ch_names])

        ## plotting
        figs = spec.plot(vmin=0.7, vmax=1.4,  cmap=parula_map, show=False, picks=[pick])

        for fig in figs:
            for ax in fig.get_axes():
                # Remove any existing labels set by MNE's plotting function
                ax.set_xlabel('')  # Clear the existing x-axis label
                ax.set_ylabel('')  # Clear the existing y-axis label
                
                # Optionally, if you want to remove the colorbar label as well, you would need to find the colorbar's axis.
                # This part might be a bit trickier and depends on how the figure is structured.
                
                # # Now, set your custom labels with your preferred fontsize
                # ax.set_xlabel('Time from stimulus onset (s)', fontsize=24)
                # ax.set_ylabel('Frequency (Hz)', fontsize=24)
                
                # # Adding a secondary y-axis for Log-Power (dB)
                # secax = ax.secondary_yaxis('right')
                # secax.set_ylabel('Log-Power (dB)', fontsize=24)
                
                # # Adjust tick label sizes
                # ax.tick_params(axis='y', labelsize=24)
                # ax.tick_params(axis='x', labelsize=24)
                # secax.tick_params(axis='y', labelsize=24)

                
        fig_path = os.path.join(layout.root, 'derivatives', 'figs', spec_type, 'picks')
        # Ensure the directory exists
        os.makedirs(fig_path, exist_ok=True)
        for i, f in enumerate(figs):
            f.savefig(os.path.join(fig_path, f'{subj}_{cond}_{i + 1}_{pick}.png'), bbox_inches='tight')
            plt.close(f)

aarons code

In [ ]:
## Description: Check channels for outliers and remove them
import mne.time_frequency
from ieeg.viz.utils import chan_grid
from ieeg.viz.parula import parula_map
from ieeg.io import get_data, update, get_bad_chans
import os
import matplotlib.pyplot as plt


## check if currently running a slurm job
HOME = os.path.expanduser("~")
if 'SLURM_ARRAY_TASK_ID' in os.environ.keys():
    LAB_root = os.path.join(HOME, "workspace", "CoganLab")
    subject = int(os.environ['SLURM_ARRAY_TASK_ID'])
else:  # if not then set box directory
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
    subject = 64
    pick = 'LAI6'


# Load the data
TASK = "SentenceRep"
layout = get_data("SentenceRep", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
conds = ["start", "aud_ls", "aud_lm", "aud_jl", "go_ls", "go_lm", "go_jl", "resp"]

for subj in subjects:
    if int(subj[1:]) != subject:
        continue
    for cond in conds:
        spec_type = 'multitaper_alt'
        filename = os.path.join(layout.root, 'derivatives',
                                'spec', spec_type, subj, f'{cond}-tfr.h5')
        try:
            spec = mne.time_frequency.read_tfrs(filename)[0]
        except OSError:
            continue
        info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])
        all_bad = get_bad_chans(info_file)
        spec.info.update(bads=[b for b in all_bad if b in spec.ch_names])

        ## plotting
        figs = spec.plot(vmin=0.7, vmax=1.4,  cmap=parula_map, show=False, picks=[pick])
        fig_path = os.path.join(layout.root, 'derivatives', 'figs', spec_type, 'picks')
        for i, f in enumerate(figs):
            f.savefig(os.path.join(fig_path, f'{subj}_{cond}_{i + 1}_{pick}.eps'), bbox_inches='tight')
            plt.close(f)



